In [ ]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib

import vis.fields
import matplotlib.cm as cm
import time

In [ ]:
# Set to True in the options below if you already optimized a specific linkage
# and you would like to reuse the optimized linkage. Loading is performed in
# place of the full design optimization.
# NOTE: Doesn't seem to work at the moment, leave at False
LOAD_OPTIMIZED_DOFS = False

# Set to False if strips cannot be labeled into families for a model
USE_FAMILY_LABEL = False

In [ ]:
# Heart Coarse 1
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1/150, 1/15]
MODEL_NAME = "heart_coarse_1"
MODEL_PATH = osp.join(weaving_dir + 'scaled_objs/models/{}.obj'.format(MODEL_NAME))
SUBDIVISION_RESOLUTION = 20
SMOOTHING_WEIGHT = 10
REGULARIZATION_WEIGHT = 0
INPUT_SURFACE_PATH = osp.join(weaving_dir + 'scaled_objs/surface_models/{}.obj'.format(MODEL_NAME))
RIBBON_NAME = "heart_coarse_1_strip"

In [ ]:
def initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH):
    l = elastic_rods.SurfaceAttractedLinkage(surface_path, useCenterline, model_path, subdivision_res, False, InterleavingType.triaxialWeave)
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    l.setDoFs(l.getDoFs())
    l.set_holdClosestPointsFixed(True);
    l.set_attraction_tgt_joint_weight(0.01);
    l.attraction_weight = 100;
    return l

In [ ]:
def initialize_normal_linkage(cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH):
    l = elastic_rods.RodLinkage(model_path, subdivision_res, False)
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    return l

In [ ]:
def design_parameter_solve(l,regularization_weight = 0.1, smoothing_weight = 1):
    design_opts = elastic_rods.NewtonOptimizerOptions()
    design_opts.niter = 10000
    design_opts.verbose = 10
    l.set_design_parameter_config(use_restLen = True, use_restKappa = True)
    elastic_rods.designParameter_solve(l, design_opts, regularization_weight = 0.0, smoothing_weight = 0.001)
    l.set_design_parameter_config(use_restLen = True, use_restKappa = True)

In [ ]:
def get_linkage_eqm(l, opt, cam_param = default_camera_parameters, target_surf = None):
    elastic_rods.compute_equilibrium(l, options = opt)
    if (target_surf is None):
        view = linkage_vis.LinkageViewer(l, width=1024, height=640)
    else:
        view = linkage_vis.LinkageViewerWithSurface(l, target_surf, width=1024, height=640)
    view.setCameraParams(cam_param)
    return l, view

In [ ]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 10;
OPTS.beta = 1e-8
OPTS.niter = 100
OPTS.verboseNonPosDef = False

### Generate straight linkage equilibrium for testing later

In [ ]:
straight_linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)

In [ ]:
view = linkage_vis.LinkageViewer(straight_linkage, width=1024, height=640)
view.setCameraParams(default_camera_parameters)
view.show()

In [ ]:
# ribbon_linkage_helper.export_linkage_geometry_to_obj(straight_linkage, '{}_intialized.obj'.format(MODEL_NAME))

In [ ]:
straight_linkage.attraction_weight = 0
with so(): straight_linkage, intial_view = get_linkage_eqm(straight_linkage, cam_param = default_camera_parameters, opt = OPTS)
intial_view.show()

In [ ]:
straight_rod_dof = straight_linkage.getDoFs()

### Create Surface Attracted Linkage and solve for design parameters with fixed joint positions

In [ ]:
import linkage_optimization

In [ ]:
with so(): linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
# with so(): linkage = initialize_normal_linkage(model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
save_tgt_joint_pos = linkage.jointPositions();

In [ ]:
with so(): design_parameter_solve(linkage, regularization_weight = 0.0, smoothing_weight = 0.001)

In [ ]:
start_time = time.time()
with so(): linkage, view = get_linkage_eqm(linkage, cam_param = default_camera_parameters, opt = OPTS, target_surf=INPUT_SURFACE_PATH)
print('compute equilibrium takes: ', time.time() - start_time)
view.show()

In [ ]:
# Optionally configure the visualization colors/transparency
view.viewOptions[view.ViewType.LINKAGE].color = 'red'
view.viewOptions[view.ViewType.LINKAGE].transparent = False
view.viewOptions[view.ViewType.SURFACE].transparent = True
view.viewOptions[view.ViewType.SURFACE].color = 'gray'
view.applyViewOptions()

In [ ]:
OPTS.niter = 200
linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
useCenterline = True
optimizer = linkage_optimization.WeavingOptimization(linkage, INPUT_SURFACE_PATH, useCenterline, equilibrium_options=OPTS, pinJoint = 0, useFixedJoint = False)
optimizer.set_target_joint_position(save_tgt_joint_pos)
view.update()

In [ ]:
optimizer.rl_regularization_weight = 0
optimizer.smoothing_weight = 10
optimizer.beta = 500000.0
optimizer.gamma = 1
algorithm = linkage_optimization.WeavingOptAlgorithm.NEWTON_CG
def update_viewer():
    view.update()

In [ ]:
if not LOAD_OPTIMIZED_DOFS:
    optimizer.WeavingOptimize(algorithm, 2000, 1.0, 1e-2, update_viewer)
else:
    loadedDoFs = np.load("dof_files/{}.npy".format(MODEL_NAME))
    linkage.setExtendedDoFs(loadedDoFs)
    view.update()

In [ ]:
## If you want to solve again after loading optimized DOFs (e.g. if the code changed)
# algorithm = elastic_rods.WeavingOptAlgorithm.NEWTON_CG
# optimizer.WeavingOptimize(algorithm, 2000, 1.0, 1e-2, update_viewer)

In [ ]:
# Store DoFs to file
if not os.path.exists("dof_files"):
    os.makedirs("dof_files")
rawDoFs = linkage.getExtendedDoFs()
np.save("dof_files/{}.npy".format(MODEL_NAME), rawDoFs)

In [ ]:
optimizer.get_linesearch_weaver().energy(elastic_rods.SurfaceAttractionEnergyType.Elastic)

In [ ]:
linkage.energy(elastic_rods.SurfaceAttractionEnergyType.Elastic)

In [ ]:
optimizer.setLinkageAttractionWeight(1e-16)
optimizer.set_holdClosestPointsFixed(False)

In [ ]:
if not LOAD_OPTIMIZED_DOFS:
    optimizer.WeavingOptimize(algorithm, 2000, 1.0, 1e-2, update_viewer)
else:
    loadedDoFs = np.load("dof_files/{}.npy".format(MODEL_NAME))
    linkage.setExtendedDoFs(loadedDoFs)
    view.update()

In [ ]:
linesearch_linkage = optimizer.get_linesearch_weaver()

In [ ]:
linesearch_linkage.energy(elastic_rods.SurfaceAttractionEnergyType.Elastic)

In [ ]:
linkage.energy(elastic_rods.SurfaceAttractionEnergyType.Elastic)

In [ ]:
start_time = time.time()
with so(): validation_linkage, validation_view = get_linkage_eqm(optimizer.get_linesearch_weaver(), cam_param = default_camera_parameters, opt = OPTS, target_surf=INPUT_SURFACE_PATH)
print('compute equilibrium takes: ', time.time() - start_time)
validation_view.show()

In [ ]:
validation_linkage.energy(elastic_rods.SurfaceAttractionEnergyType.Elastic)